### Zynab Smaan

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, to_timestamp

spark = SparkSession \
    .builder \
    .appName("Analyzing New york city trips") \
    .getOrCreate()

In [33]:
trips = spark.read\
               .format("csv")\
               .option("header", "true")\
               .load("sample.csv")

In [34]:
trips.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [35]:
# Let's drop these columns
trips = trips.drop('medallion', 'vendor_id', 'rate_code', 'store_and_fwd_flag')
trips.columns

['hack_license',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_time_in_secs',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude']

In [36]:
trips.show(1)

+--------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              4|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|
+--------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
only showing top 1 row



In [37]:
trips.printSchema()

root
 |-- hack_license: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [38]:
trips.count()

99999

In [39]:
# Drop nan values.
trips = trips.dropna()
trips.count()

99999

In [68]:
# Create column duration 
trips = trips.withColumn("duration", unix_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss")
                         .cast("double") - unix_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss")
                         .cast("double"))

In [69]:
trips.show(5)

+--------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+--------+
|        hack_license|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|duration|
+--------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+--------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              4|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|   382.0|
|9FD8F69F0804BDB55...|2013-01-06 00:18:35|2013-01-06 00:22:54|              1|              259|         1.50|      -74.006683|      40.731781|       -73.994499|        40.75066|   259.0|
|9FD8F69F0804BDB55...|2013-01-05 18:49:41|2013-01-05 18:54:2

In [75]:
# filter data if the duration is less than 0 or bigger than (4 hours = 4*60*60 = 14400)
trips = trips.filter(trips['duration'] >= 14400 | trips['duration'] <= 0)

Py4JError: An error occurred while calling o251.or. Trace:
py4j.Py4JException: Method or([class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [70]:
382/60

6.366666666666666